In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo
import json

from utils import get_config


/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [83]:
downloadversion = 5

In [76]:
config = get_config()

date = '2018-12-09'
username = config['username']
marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = 'logs/'
slackfilepath = config['slackfilepath']


In [84]:
path = os.path.join(slackfilepath, "Summary_%s.csv"%(date))
if downloadversion>0:
        path = re.sub('.csv', ' ({0}).csv'.format(downloadversion),path)
df_sum = pd.read_csv(path)
path = os.path.join(slackfilepath, "Photometry_%s.csv"%(date))
if downloadversion>0:
        path = re.sub('.csv', ' ({0}).csv'.format(downloadversion),path)
df_phot = pd.read_csv(path)


FileNotFoundError: File b'/home/jnordin/Downloads/Summary_2018-12-09 (5).csv' does not exist

In [78]:
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)


INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 1041 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-12-12 13:03:44 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration number 0: 58 jobs to do
INFO:marshaltools.ProgramList:jobs are done: total 58, failed: 0
INFO:marshaltools.ProgramList:Fetched 4299 candidates in 2.58e+01 sec


In [79]:
ingestoldcand_start = Time.now()
toingest = []
for sne in df_sum.iterrows():
    print( sne[1]["ztf_name"] )
    name = sne[1]["ztf_name"]
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    
            
    if name in pl.sources:
        print("%s already saved in cosmology!"%(name))
        continue
        
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==name ]  )
    print("Trying to ingest %s through avroid %s"%(name,avroid))
    canddate = pl.find_source(name,include_candidates=True)
    if not canddate is None:
        print('... already a candidate with id %s'%(canddate['candid']))
        continue
    toingest.append(avroid)

    
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
failed = pl.ingest_avro(toingest)
print("Tried to ingest %s, failed for %s"%(toingest,failed))
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

INFO:marshaltools.ProgramList:Trying to ingest 9 candidate(s) to to marhsal program AMPEL Test using ingest ID 4


ZTF18aaeqget
ZTF18aaeqget already saved in cosmology!
ZTF18aajhfdm
ZTF18aajhfdm already saved in cosmology!
ZTF18acdvuzj
Trying to ingest ZTF18acdvuzj through avroid 707481974315015026
ZTF18acrdlrp
ZTF18acrdlrp already saved in cosmology!
ZTF18acrdmmw
ZTF18acrdmmw already saved in cosmology!
ZTF18acrhiia
ZTF18acrhiia already saved in cosmology!
ZTF18acrxnme
Trying to ingest ZTF18acrxnme through avroid 707447905115015017
ZTF18acrxppd
Trying to ingest ZTF18acrxppd through avroid 707464225515015014
ZTF18acrxpxd
Trying to ingest ZTF18acrxpxd through avroid 707467950515015005
ZTF18acrxpzx
Trying to ingest ZTF18acrxpzx through avroid 707467955415015014
ZTF18acrxtrv
Trying to ingest ZTF18acrxtrv through avroid 707470383715015027
ZTF18acrxtul
Trying to ingest ZTF18acrxtul through avroid 707470386015015011
ZTF18acrxwxm
ZTF18acrxwxm already saved in cosmology!
ZTF18acsonov
ZTF18acsonov already saved in cosmology!
ZTF18acsoptf
ZTF18acsoptf already saved in cosmology!
ZTF18acsotqw
ZTF18acsotqw alr

INFO:marshaltools.ProgramList:Attempt 1: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-12-11 13:04:10.706 and 2018-12-12 13:14:34.198
INFO:marshaltools.ProgramList:attempt # 1. Of the desired candidates 4 successfully ingested, 5 failed
INFO:marshaltools.ProgramList:Attempt 2: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-12-11 13:04:10.706 and 2018-12-12 13:14:51.943
INFO:marshaltools.ProgramList:attempt # 2. Of the desired candidates 0 successfully ingested, 5 failed
INFO:marshaltools.ProgramList:Attempt 3: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-12-11 13:04:10.706 and 2018-12-12 13:15:08.776
INFO:marshaltools.ProgramList:attempt # 3. Of the desired candidates 0 successfully ingested, 5 failed


Tried to ingest [707481974315015026, 707447905115015017, 707464225515015014, 707467950515015005, 707467955415015014, 707470383715015027, 707470386015015011, 707480125515015003, 707477333515015018], failed for ['707467950515015005', '707467955415015014', '707470383715015027', '707470386015015011', '707480125515015003']
Stoppedingesting at 2018-12-12 13:05:12.794003 (useful if you want to check scanning page directly)


In [80]:
# Ingested, now try saving
tosave = []
for sne in df_sum.iterrows():
    print( sne[1]["ztf_name"] )
    name = sne[1]["ztf_name"]
            
    if name in pl.sources:
        print("%s already saved in cosmology!"%(name))
        continue
        
    tosave.append(name)
    pl.save_source_name(name,42)

print('Tried to save:')
print(tosave)
    


INFO:marshaltools.ProgramList:Saving source ZTF18acdvuzj into program 42


ZTF18aaeqget
ZTF18aaeqget already saved in cosmology!
ZTF18aajhfdm
ZTF18aajhfdm already saved in cosmology!
ZTF18acdvuzj


INFO:marshaltools.ProgramList:Saving source ZTF18acrxnme into program 42


ZTF18acrdlrp
ZTF18acrdlrp already saved in cosmology!
ZTF18acrdmmw
ZTF18acrdmmw already saved in cosmology!
ZTF18acrhiia
ZTF18acrhiia already saved in cosmology!
ZTF18acrxnme


INFO:marshaltools.ProgramList:Saving source ZTF18acrxppd into program 42


...

ZTF18acrxtul


INFO:marshaltools.ProgramList:Saving source ZTF18acueeoo into program 42


ZTF18acrxwxm
ZTF18acrxwxm already saved in cosmology!
ZTF18acsonov
ZTF18acsonov already saved in cosmology!
ZTF18acsoptf
ZTF18acsoptf already saved in cosmology!
ZTF18acsotqw
ZTF18acsotqw already saved in cosmology!
ZTF18acsouto
ZTF18acsouto already saved in cosmology!
ZTF18acsouxk
ZTF18acsouxk already saved in cosmology!
ZTF18acsovsw
ZTF18acsovsw already saved in cosmology!
ZTF18acsowmi
ZTF18acsowmi already saved in cosmology!
ZTF18acsozcp
ZTF18acsozcp already saved in cosmology!
ZTF18acsptko
ZTF18acsptko already saved in cosmology!
ZTF18acueemg
ZTF18acueemg already saved in cosmology!
ZTF18acueeoo


INFO:marshaltools.ProgramList:Saving source ZTF18acurkie into program 42


ZTF18acuqxyf
ZTF18acuqxyf already saved in cosmology!
ZTF18acurkie
Tried to save:
['ZTF18acdvuzj', 'ZTF18acrxnme', 'ZTF18acrxppd', 'ZTF18acrxpxd', 'ZTF18acrxpzx', 'ZTF18acrxtrv', 'ZTF18acrxtul', 'ZTF18acueeoo', 'ZTF18acurkie']


In [81]:
# Check whether they were saved
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)


INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 1050 saved sources for program AMPEL Test.


In [82]:
for name in tosave:
    if name in pl.sources:
        print("%s already saved"%(name))
        continue
        
    print('no? %s'%(name))

ZTF18acdvuzj already saved
ZTF18acrxnme already saved
ZTF18acrxppd already saved
ZTF18acrxpxd already saved
ZTF18acrxpzx already saved
ZTF18acrxtrv already saved
ZTF18acrxtul already saved
ZTF18acueeoo already saved
ZTF18acurkie already saved


In [20]:
# First set of failed avros
tried1 = [708276792215015012, 708273146215015014, 708279084015015061, 708299946115015010, 708272683815015008, 708276795015015000, 708277704015015028, 708277705215015014, 708276342515015013, 708275873415015000, 708275423415015005, 708275422415015017, 708274510615015000, 708274512615015006]
failed1 =  ['708276792215015012', '708273146215015014', '708299946115015010', '708272683815015008', '708276795015015000', '708275423415015005', '708275422415015017', '708274510615015000', '708274512615015006']
#Stoppedingesting at 2018-12-12 12:14:07.950674 (useful if you want to check scanning page directly)
failed2 = ['708264276015015019', '708264270515015003']
failed3 = ['707480125315015020', '707512243615015036', '707535943115015041', '707505253515015007', '707541632715015009', '707505705315015002']
['707257983915015016', '707259350715015008', '707502965115015016']
failed for ['707541143215015015', '707529403815015011', '707536414315015036', '707541633215015002', '707542566115015004', '707545313115015031', '707546225815015000', '707543020415015010', '707541632315015002', '707541631415015018', '707530853015015019', '707530854215015005', '707533620415015026', '707542083215015001', '707538344815015017', '707547142415015006']
'707523500615015001', '707523501815015021', '707505701115015029', '707488855815015012', '707518784415015004', '707493021915015000', '707523506015015028', '707512245315015043', '707517832915015003', '707519713715015002', '707494032015015004', '707514096115015012', '707514543115015004', '707514542815015016', '707502045315015004', '707523002815015072', '707495431115015001', '707494032115015007', '707505701815015010', '707518780915015003', '707513170115015028', '707519710015015005', '707506165215015023', '707498194215015011', '707503422115015009']

In [ ]:
no? ZTF18acsotcf
no? ZTF18acsotqw
no? ZTF18acsovsw

In [ ]:
Trying to ingest ZTF18acsotcf through avroid 707502045315015004
ZTF18acsotqw
Trying to ingest ZTF18acsotqw through avroid 707503423715015034
ZTF18acsovsw
Trying to ingest ZTF18acsovsw through avroid 707523002815015072